In [1]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_simulation_baseline as file

In [2]:
# Args setup
ntrial = 50
n = 500
n_int = 100
niter = 1000
front_door = True
minimise = False

In [3]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="3:0:0",
    
    job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

/nfs/ghome/live/danceh/.local/mambaforge/envs/causalgp/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 11111 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33399 instead
  warnings.warn(


In [4]:
# Submitting jobs
futures = []
for seed in range(ntrial):
        f = client.submit(file.main,
                              seed,
                              n,n_int,
                              niter = niter,
                              front_door = front_door,
                              minimise = minimise
                             )
        futures += [f]

In [5]:
futures

[<Future: pending, key: main-1c02f02e7f6d446a0f5dbccd49ffcdd0>,
 <Future: pending, key: main-fc431fcbe84d08612c56cc8d34d03277>,
 <Future: pending, key: main-ac2b77c2f207f53d7a99448cfc039c8f>,
 <Future: pending, key: main-45bf24b81f49f1caac6fe2bf3f4697f0>,
 <Future: pending, key: main-a5da9861f5d0dfc695bdaa59c715cd3f>,
 <Future: pending, key: main-d754103a90352b1dd695c397b6409dc2>,
 <Future: pending, key: main-e1f2c0794babf1f25f1cc20ba1dc2dd5>,
 <Future: pending, key: main-057fd281dfd17be9ef5e9e393c31a11f>,
 <Future: pending, key: main-699bce4d1349e315eb1e6ccaa373d3ab>,
 <Future: pending, key: main-29eee1d48cce351145d568181a48795f>,
 <Future: pending, key: main-67ebd7800d8e6cb8cc55af25d24c4c0e>,
 <Future: pending, key: main-e68e64bdba0fefb71998cdf0a6608dea>,
 <Future: pending, key: main-1903148c8a5ae62d8813827a8732259f>,
 <Future: pending, key: main-0f218ba1534edc462abc027d8f1b582c>,
 <Future: pending, key: main-c724ed70a5475883bc47016dc1040a4f>,
 <Future: pending, key: main-666d073d7c3

In [6]:
# Getting results
results = client.gather(futures)

In [7]:
# Closing client
client.close()
cluster.close()

In [8]:
# Saving results
torch.save(obj = results,
           f = "simulation_baseline_ntrial={0}_n={1}_frontdoor={2}_minimise={3}.pt".format(ntrial,
                                                                                             n,
                                                                                             front_door,
                                                                                             minimise)
          )